In [21]:
import pyrealsense2 as rs
import numpy as np
import cv2
import model
    

In [22]:
def most_frequent(List):
    
    if len(List) != 0:
        counter = 0
        value = List[0]
     
        for i in List:
            curr_frequency = List.count(i)
            if(curr_frequency> counter):
                counter = curr_frequency
                value = i
    
        return value
    else:
        return 0


def make_bounding_box(filtered_faces,filtered_labels,max_appeared_label,frame):
    for per_person in range(max_appeared_label):
            per_person_face = filtered_faces[:,per_person]
            per_person_emotion = filtered_labels[:,per_person]
        
            final_label = most_frequent([i for i in per_person_emotion])
            x,y,w,h = per_person_face[np.where(per_person_emotion == final_label)][0]
            
            label_position = (x,y)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,145,255),1)
            cv2.putText(frame,final_label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

In [23]:



class DepthCamera:
    def __init__(self):
        # Configure depth and color streams
        self.pipeline = rs.pipeline()
        config = rs.config()

        # Get device product line for setting a supporting resolution
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)



        # Start streaming
        self.pipeline.start(config)

    def get_frame(self):
        frames = self.pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        if not depth_frame or not color_frame:
            return False, None, None
        return True, depth_image, color_image

    def release(self):
        self.pipeline.stop()

In [24]:
# Initialize Camera Intel Realsense
dc = DepthCamera()
cv2.namedWindow("Color frame")


# while True:
#     ret, depth_frame, color_frame = dc.get_frame()
    

#     cv2.imshow("depth frame", depth_frame)
#     cv2.imshow("Color frame", color_frame)
#     key = cv2.waitKey(1)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break



while True:
    
            tot_labels_set = []
            tot_faces_set = []

            frame = None

            val = True
            
            for frm_of_set in range(30):
                val,depth_frame,color_frame = dc.get_frame()
                if val :
                    labels,faces = model.Model.fer(color_frame)

                    if labels != []:
                        tot_labels_set.append(np.array(labels))
                        tot_faces_set.append(np.array(faces))
            

            if val :
                    tot_faces_set = np.array(tot_faces_set)
                    tot_labels_set = np.array(tot_labels_set)

                    max_appeared_label = most_frequent([len(i) for i in tot_labels_set ])

                    #filtered according to the max no appeared faces found in each frame to filter 
                    filtered_indexes = [index for index in range(len(tot_labels_set)) if len(tot_labels_set[index])==max_appeared_label]

                    filtered_labels= np.array([tot_labels_set[index] for index in filtered_indexes ])
                    filtered_faces = np.array([tot_faces_set[index] for index in filtered_indexes ])

                    if max_appeared_label >= 1:  
                    
                        make_bounding_box(filtered_faces,filtered_labels,max_appeared_label,frame)

                        cv2.imshow('Emotion Detector',frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break
                        else:
                            count = count+1
                            continue
                        
                    else:

                        cv2.putText(frame,'No Faces',(179,59),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
                        cv2.imshow('Emotion Detector',frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break
                        else: 
                            count = count+1
                            continue
            else:
                break        
                        
                        
    
print(count)
cv2.destroyAllWindows()    

TypeError: fer() missing 1 required positional argument: 'frame'